In [ ]:
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial import distance

# Function to estimate the number of clusters based on image complexity
def estimate_clusters_based_on_image(image_array, base_clusters=5):
    num_pixels = image_array.size // 3
    estimated_clusters = min(base_clusters + (num_pixels // 10000), 50)  # Adjust as needed
    return estimated_clusters

# Function to find the closest color in the CSV
def closest_color(rgb, color_df):
    distances = color_df.apply(lambda row: distance.euclidean(rgb, [row['Red (8 bit)'], row['Green (8 bit)'], row['Blue (8 bit)']]), axis=1)
    closest_index = distances.idxmin()  # Get the index of the closest color
    return color_df.loc[closest_index]

#Load the Image
image_path = 'img1.jpeg'
image = Image.open(image_path)
image = image.resize((image.width // 10, image.height // 10))  # Resize for faster processing

# Convert Image to RGB Values
image_array = np.array(image)
pixels = image_array.reshape(-1, 3)  # Reshape to list of pixels

#Estimate the number of clusters
n_colors = estimate_clusters_based_on_image(image_array)
print(f"Estimated number of clusters: {n_colors}")

#Cluster Colors
kmeans = KMeans(n_clusters=n_colors)
kmeans.fit(pixels)
dominant_colors = kmeans.cluster_centers_.astype(int)

# Load the CSV Data
color_df = pd.read_csv('color_names.csv')

# Match Dominant Colors to Closest Colors in CSV
matched_colors = [closest_color(color, color_df) for color in dominant_colors]

#  Display the Dominant Colors and Their Closest Match
plt.figure(figsize=(10, 6))
for i, color_info in enumerate(matched_colors):
    rgb_value = [color_info['Red (8 bit)'], color_info['Green (8 bit)'], color_info['Blue (8 bit)']]
    color_name = color_info['Name']
    
    plt.subplot(1, n_colors, i + 1)
    plt.imshow([[rgb_value]])
    plt.axis('off')
    plt.title(color_name)

plt.show()

# Optionally print out the RGB values and color names
for i, color_info in enumerate(matched_colors):
    print(f"Dominant Color {i + 1}: {color_info['Name']} (RGB: {color_info[['Red (8 bit)', 'Green (8 bit)', 'Blue (8 bit)']].values})")
